In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
pip install rouge


In [14]:
!pip install nltk
!pip install rouge_score



  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=4c4f1806fe97d790eb06cc31d7f958507acbe10c9ee34edc42d604e4c90acc90
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


# **TOPIC MODELLING + LDA**

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim import corpora, models
from sklearn.feature_extraction.text import TfidfVectorizer
import networkx as nx
from gensim.models import CoherenceModel
import re

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    sentences = sent_tokenize(text)
    preprocessed_sentences = []
    for sentence in sentences:
        words = [word.lower() for word in word_tokenize(sentence) if word.isalpha() or word == '.']
        preprocessed_sentences.append(' '.join(words))  # Join words into a single string
    return preprocessed_sentences

def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def lda_topic_modeling(texts, num_topics=2):
    dictionary = corpora.Dictionary([text.split() for text in texts])  # Split text into words
    corpus = [dictionary.doc2bow(text.split()) for text in texts]  # Split text into words

    lda_model = models.LdaModel(corpus, num_topics=num_topics)
    topics = lda_model[corpus]

    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    return lda_model, topics, corpus, dictionary, coherence_lda

def text_rank_summarization(text_list, max_sentence_length=15, max_sentences=1000):
    text = ' '.join(text_list)  # Combine the list of sentences into a single string
    text = re.sub(r'\.', '. ', text)  # Add space after period for proper sentence splitting
    sentences = sent_tokenize(text)

    # Create summary by joining sentences with full stops every max_sentence_length words
    summary = ''
    num_sentences = 0
    for i, sentence in enumerate(sentences):
        words_in_sentence = sentence.split()
        if num_sentences + len(words_in_sentence) // max_sentence_length > max_sentences:
            break
        if len(words_in_sentence) > max_sentence_length:
            words_in_sentence = words_in_sentence[:max_sentence_length]  # Limit sentence length
        if i < len(sentences) - 1:
            if num_sentences < max_sentences:
                summary += ' '.join(words_in_sentence) + '. '  # Add full stop at the end of each sentence
                num_sentences += 1
        else:
            summary += ' '.join(words_in_sentence) + '. '  # Add full stop at the end of last sentence

    return summary

# Example usage with a text file
file_path = '/content/drive/MyDrive/validation (2)/annual_reports (1)/30777.txt'
document = read_text_file(file_path)

# Preprocess text
processed_sentences = preprocess_text(document)

# Topic modeling with LDA
num_topics = 20
lda_model, topics, corpus, dictionary, coherence_lda = lda_topic_modeling(processed_sentences, num_topics=num_topics)

# Extractive summarization using TextRank with custom sentence length and max sentences
summary = text_rank_summarization(processed_sentences, max_sentence_length=15, max_sentences=1000)

# Print the summary
print("Summary:")
print(summary)


def correct_full_stops_and_capitalize(summary):
    # Replace all occurrences of double full stops with single full stops
    corrected_summary = summary.replace("..", ".")

    # Capitalize the first letter of each sentence
    sentences = corrected_summary.split(". ")
    corrected_sentences = [sentence.strip().capitalize() for sentence in sentences]

    # Join the corrected sentences back together
    corrected_summary = ". ".join(corrected_sentences)

    # Ensure the last character is a full stop
    if not corrected_summary.endswith("."):
        corrected_summary += "."

    return corrected_summary

corrected_summary = correct_full_stops_and_capitalize(summary)

from IPython.display import display, Markdown
display(Markdown(corrected_summary))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


Summary:
march pm proof march pm proof annual report bodycote plc annual report stock code boy. experienced in supporting large multinational customers and their supply chains as well as local niche. the core values underpinning everything we do honesty and t ransparency we are honest and. this is not something we take for granted .. bodycote lives by a culture of honest and transparent behaviour which is at the core. respect and responsibility we manage our business with respect applying an ethical approach to our. we believe in taking ownership and being mindful of the impact of our actions .. creating v alue creating value is the very essence of our business and needs to. we create value for our customers our employees and our shareholders .. throughout this report you will see illustrations which link our business and strategy strategy core. headline operating profit and headline profit before taxation exclude amortisation of acquired intangibles of and. .. return on sales is defin

March pm proof march pm proof annual report bodycote plc annual report stock code boy. Experienced in supporting large multinational customers and their supply chains as well as local niche. The core values underpinning everything we do honesty and t ransparency we are honest and. This is not something we take for granted. Bodycote lives by a culture of honest and transparent behaviour which is at the core. Respect and responsibility we manage our business with respect applying an ethical approach to our. We believe in taking ownership and being mindful of the impact of our actions. Creating v alue creating value is the very essence of our business and needs to. We create value for our customers our employees and our shareholders. Throughout this report you will see illustrations which link our business and strategy strategy core. Headline operating profit and headline profit before taxation exclude amortisation of acquired intangibles of and. . Return on sales is defined as headline operating profit as a percentage of revenue. . A detailed eps reconciliation is provided in note on page. . Return on capital employed roce is defined as headline operating profit of divided by the. Capital employed is defined as net assets adjusted for net debt. . Headline operating cash flow is defined as cash generated by operations of less net capital. . Headline operating cash conversion is defined as headline operating cash flow divided by headline operating. . Free cash flow is defined as cash generated by operations of less net capital expenditure. Bodycote plc annual report for the year ended december march pm proof strategic report the. The report outlines the developments and performance of the group during the financial year the. Key performance indicators are published to show the performance and position of the group. Page outlines the group s strategy and objectives along with the business model on page. The directors in preparing this strategic report have complied with of the companies act. This strategic report has been prepared for the group as a whole and therefore gives. The strategic report discusses the following areas global network chairman s statement chief executive s. Through an international network of plants bodycote can effectively utilise a wealth of knowledge experience. The network operates from facilities with customers able to benefit from bodycote s comprehensive range. Customers know that if their business expands bodycote will have the capability to meet their. They recognise that if they were to broaden their manufacturing footprint bodycote would be able. They are aware that they can obtain the same process to the same quality standards. Such a large network brings economies of scale with technology developed at one location being. Similarly network utilisation is enhanced by using logistics to put customers work into the most. The bodycote network has a wealth of technical accreditations some industry or customer specific others. Individual operations concentrate on the accreditations suited to their market. Global network group revenue by market sector aerospace and defence automotive general industrial energy total. Revenue by geography bodycote plc annual report for the year ended december march pm proof. Emerging markets bodycote has facilities in emerging markets covering eastern europe china and mexico. Bodycote is the number one thermal processing provider in eastern europe and is the leading. Revenue by market sector emerging markets aerospace and defence automotive general industrial energy total although. With facilities in countries bodycote is truly global. North america revenue by market sector north america aerospace and defence automotive general industrial energy. This network offers facilities convenient to customers in all areas where manufacturing and technical industries. Strategic report governance financial statements additional information stock code boy march pm proof quinn cbe. I have spent time with all of my board colleagues and am confident that i. Quinn cbe chairman i am pleased to be contributing to the annual report for the. Overview the group delivered a good set of results for and i am very pleased. Dividend the board is proposing a final dividend of an increase of which will be. This brings the total ordinary dividend for to costing which represents a increase of. This increase in the level of dividend underscores the board s view of the group. Furthermore recognising the strong net cash position of the group at year end the board. Board and governance my predecessor alan thomson stepped down as chairman at the end of. On behalf of the board and the company as a whole i thank alan for. I think that it is fair to say that bodycote has undergone a wholesale transformation. One of the key roles of the chairman is to ensure that the board members. I have spent time with all of my board colleagues and am confident that i. Bodycote plc annual report for the year ended december march pm proof the board has. Lili is an experienced strategy consultant and since a global partner in the london office. Lili currently heads the oil gas practice and consults to a range of international clients. Lili brings a wide range of experience in businesses including those in the engineering aviation. Another one of the key responsibilities of the chairman is to promote effective governance across. We are committed to conducting business responsibly. By maintaining high standards of corporate governance we enhance performance underpinned by our business model. Our approach to governance is set by the board and our executive committee ensures that. Effective and robust governance remains central to the ongoing success of the group. People as part of my induction to the business i have had the chance to. I have been impressed by the commitment knowledge and passion that i have experienced from. I can see how the talented workforce creates a clear competitive advantage for bodycote which. Shareholders during the year as usual meetings were undertaken with a number of bodycote s. I have already had the opportunity to meet some of our shareholders and look forward. Summary the group is starting to show the results of the increased investment programme that. This programme has been designed to generate superior growth in what until recently has been. The results of these investments together with the continued diligence in operational efficiency and a. Bodycote remains well placed to capitalise on the value generating investments available to it and. I am confident that in the coming years bodycote will deliver an attractive return for. Quinn cbe chairman march strategic report governance financial statements additional information stock code boy march. The group s revenue growth combined with continued discipline on costs helped lift headline operating. Return on sales increased to from. Harris group chief executive overview bodycote reported revenue growth of to with revenue benefiting from. At constant currency revenues grew including a contribution of from acquisitions completed in. The following review reflects constant currency growth rates unless stated otherwise. General industrial markets returned to growth after a year negative trend. Moreover this growth was with improvements in general industrial demand occurring in all of our. Group revenues generated from the general industrial sectors which represent some of our business grew. Of this growth percentage points came from acquisitions made in. The remainder of the group s outperformance in general industrial versus the market came from. The decision we took to preserve the capacity at our t facilities is being well. Growth in onshore oil gas saw a strong sequential increase in driven predominantly by demand. In the rest of the energy sector subsea revenues continued to decline and industrial gas. It is worth noting however that requests for quotation in the subsea sector have picked. In total revenues from the energy sector increased by. Civil aviation grew with our uk operations continuing to add significant business. This increase will require new facilities to be built in in the uk in order. North america also started to contribute to the growth in a more meaningful way as. Bodycote has a much stronger position on the leap series than it did on the. Bodycote plc annual report for the year ended december march pm proof the automotive business. This compares to background demand growth in europe of low single digits and a slight. Our strong performance benefited from the contribution from the acquisitions made in together with the. In our specialist technologies we achieved revenue growth across our specialty stainless steel processes s. However several factors dragged on overall sales growth during the year. The hip product fabrication hip pf and surface technology businesses are focused largely on oil. In addition the weakness in igt volumes resulted in a slowdown in the hip services. Also impacting our hip services business was an unplanned outage during the second half which. Forecast growth in all the specialist technologies looks strong. The five sites we acquired in are performing well. They are all classical heat treatment sites within the agi division and contributed of revenue. It is also worth highlighting that revenues at these facilities have accelerated since coming into. With careful cost discipline in the face of growing revenues the group s headline operating. Once again we increased our prices ahead of cost inflation. This is an area of heightened focus for the group especially as we are entering. It is worth noting however that bodycote typically performs well in higher inflationary environments. The group s strong profit improvement coupled with a headline tax rate of increased basic. Basic earnings per share increased to from. The return on capital employed rose in the current year to from in. Free cash flow increased to as a result of our improved profitability and in spite. Headline operating cash conversion was once again above yielding a net cash position at the. Strategic progress the group s strategy encompasses the drive for operational efficiency and improvement in. The group has a minimum hurdle rate return when looking at investments. During we made further progress against our strategy delivering higher group return on sales of. We believe there is still the opportunity to further improve from both management led initiatives. In we continued to invest in areas with superior growth potential with a deliberate bias. We will continue to invest to support the future of our business in these rapid. During the year we commissioned several new lpc and s p lines. In hip services we acquired the hip assets from doncasters group limited s uk business. Further new facilities are expected to be commissioned in for both specialist technologies and classical. The additional capacity that will come on stream in will add to our ability to. We also continue to look at acquisition and investment opportunities that will grow our business. These are traditionally small facilities that can provide us with infills to our existing network. Where opportunities to buy such facilities do not exist we will build new facilities these. Since we have invested in both acquisitions and investment for growth in new and existing. Organisation and people bodycote is a service business and service is delivered by passionate and. We will continue to invest in training and developing our employees to ensure that our. Summary and outlook has once again demonstrated the quality of bodycote s business. Strong growth was achieved through contributions from contract wins on automotive and aerospace programmes excellent. The group s revenue growth combined with continued discipline on costs helped lift headline operating. Return on sales increased to from. To ensure that the business continues to deliver good results we will continue to focus. Our business by its nature has limited forward visibility but we have entered the year. Accordingly and in spite of the foreign exchange headwind at current exchange rates has started. Harris group chief executive march strategic report governance financial statements additional information stock code boy. Bodycote s classical heat treatments describe a group of mature heat treatment processes and includes. Virtually every type of metal component whatever its application has received some form of processing. Bodycote s specialist technologies refer to a group of processes which require very specialist expertise. These technologies some of which are proprietary offer unique solutions for a variety of applications. The global leader customer focus global network transferable bodycote is focused on continual improvement of. Bodycote seeks to secure specific arrangements with our customers which provide protection from supply disruption. Bodycote s global network of facilities see pages and in countries brings economies of scale. This makes bodycote s processing inherently more efficient than customers operations see page and competitors. The capital intensive nature of bodycote s business also provides significant barriers to entry. The scope of bodycote s network enables us to specialise more effectively than competitors at. The global bodycote network provides unique opportunities for the transfer of knowledge and skills and. With some of the best metallurgists engineers and technicians in the industry bodycote is ideally. Bodycote s scale enables continuous yet focused investment both in the latest processes and in. The supplier of choice service quality expertise bodycote has become the supplier of choice for. Bodycote s quality management systems validated by major engineering oems have been developed to meet. All bodycote facilities hold industry and customer approvals appropriate to the services they offer and. Bodycote s extensive facilities and expertise mean that projects can extend beyond customers capabilities combining. Our own enhancements and improvement of standard processes have led to bodycote offering a range. Creating value for customers for bodycote for investors services. Global supplier which can meet multiple processing needs. Access to entire bodycote knowledge base and expertise. Cost and environmental benefits versus operations. Mutually beneficial customer relationships. Wide customer base means bodycote is not reliant on any one customer. Ideally positioned to promote growth in emerging markets and selected technologies. Clearly focused strategy. Financially stable and sustainable business. Good growth drivers. Superior return on investment. Strong margins and cash flow. Bodycote plc annual report for the year ended december march pm proof strategy and objectives. Specialist t echnologies capitalising on our specialist technologies to provide our customers with the ability. Bodycote s objective is to create superior shareholder returns through the provision of selected thermal. The ade divisions serving the aerospace defence and energy customers with a focused network of. Emerging markets expanding with our customers to rapid growth countries with an emphasis on eastern. Operational improvement continuous improvement of business processes and systems which make us more efficient and. Acquisitions adding small acquisitions to improve our plant network in classical heat t reatment and. The agi divisions serving the automotive and general industrial customers through a regionally organised business. Divisional customer focus organising our technologies and resources to align with the global and local. T echnology providing thermal processing services that are a vital link in the manufacturing supply. Strategic elements our key strategies focusing on customer alignment and leadership in our areas of. Hip services hip product fabrication surface t echnology return on capital employed headline earnings per. Strategic report governance financial statements additional information stock code boy march pm proof component journeys. Materials used are customised based on the end application including high temperature stainless steels and. Denotes the parts of the component journey undertaken by bodycote. Bodycote component journeys this is just one example of how bodycote brings together the huge. For more component journeys visit bearings are shaped and machined from rods of metal plates. For the perfect shape and smooth finish bearings are put through final lapping and polishing. The surface area to be heat treated is prepared areas which are not to be. The surface is hardened via a specialised nitriding process during which nitrogen is diffused into. After heat treatment the masking is removed and the surface cleaned before inspection to ensure. End applications include landing gear engine control surfaces and other parts or components. Bodycote plc annual report for the year ended december march pm proof inner strength a. Our bones effectively composites absorb such stresses regularly and effectively for much of our lifetime. When joints fail they are often replaced with metal alloy implants. These implants must be incredibly strong biocompatible and able to last the lifetime of the. A combination of heat treatment hot isostatic pressing and coating makes this possible. Denotes the parts of the component journey undertaken by bodycote. Bodycote component journeys this is just one example of how bodycote brings together the huge. For more component journeys visit cobalt chromium alloy billets are investment cast to form implant. The castings are thermally sprayed with a biomedical coating to allow a bond to form. The implants are then hiped to eliminate porosity improve fatigue life and enhance the bonding. Solution and ageing heat treatment is used to strengthen the implant. End application joint replacement. Strategic report governance financial statements additional information stock code boy march pm proof in gear. During use a vehicle places heavy demand on its transmission requiring a fast and reliable. The gears require high strength and wear resistance in order to withstand the stresses applied. Bodycote s heat treatment processes in particular low pressure carburising lpc enable modern transmissions to. Denotes the parts of the component journey undertaken by bodycote bodycote component journeys this is. The gears are assembled into the transmission unit. The gears are dimensionally measured before heat treatment to monitor and maintain repeatability of distortion. The gears are then heat treated using lpc to enhance functionality by adding a case. The gears are quenched using nitrogen gas to minimise part distortion then tempered to relieve. The gears are machined to shape using a shaving or hobbing method. The gears are shot peened to add residual stress this allows the parts to withstand. The gears are measured again after heat treatment to check any distortion is within limits. The parts are inspected and tested for surface hardness core hardness and effective case depth. End application automobile. Bodycote plc annual report for the year ended december march pm proof our technologies classical. These tightly controlled processes condition the material properties including both the core properties and the. A set of differentiated processes giving customers the ability to produce unique high products. Hip hot isostatic pressing services hip impact resistance and fatigue properties are extremely sensitive to. Through the simultaneous application of heat and pressure the hip process eliminates internal porosity improving. Surface hardness what is it the ability of a material to resist deformation scratching and. Why is it important improving a material s hardness through heat treatment allows it to. Hip pf hot isostatic pressing product fabrication hip pf this method combines the hip process. The flexibility of the hip pf process means that combinations of materials can be used. T oughness what is it the ability of a material to absorb energy and plastically. Why is it important heat treatment can be used to strengthen the material and help. S p specialty stainless steel processes s p steel is often chosen for its inherent. Standard heat treatments will harden the steel but can negatively impact the corrosion resistance. S p technology uniquely hardens stainless steel alloys and alloys improving mechanical and wear properties. Fatigue strength what is it the stress level at which component failure occurs when subjected. Why is it important part failure due to fatigue can have catastrophic consequences particularly if. Through heat treatment a material s fatigue strength is improved. Lpc low pressure carburising lpc a case hardening process used to obtain a hardened surface. Lpc is a clean process carried out under vacuum and is an environmentally friendly treatment. Creep resistance what is it the measure of a material s ability to resist high. Why is it important some metals and alloys must operate at temperatures close to their. Heat treatment enables them to perform at higher temperatures with little or no movement. Cid cid a proprietary thermochemical treatment for the simultaneous improvement of corrosion and wear resistance. Is an environmentally friendly alternative to the use of hard chromium electroless nickel and other. Core ductility what is it the ability of a material to deform without breaking. Why is it important in order to form or shape a complex component good ductility. Heat treatment is used to soften the material which makes it easy to work as. St surface t echnology st st incorporates specialised plasma spray high velocity oxygen fuel hvof. Below are a few examples of material properties obtained by heat treatment strategic report governance. Headline operating profit increased by from to while average capital employed increased by to. Definition headline operating profit as a percentage of the average of opening and closing capital. Capital employed is defined as net assets adjusted for net debt. Performance headline earnings per share increased by pence during the year from pence to pence. Definition headline earnings per share is defined in note to the financial statements. Performance return on sales increased by percentage points during the year from to. Headline operating profit increased by from to while revenue increased by from to. Definition headline operating profit as a percentage of revenue. Bodycote plc annual report for the year ended december march pm proof headline operating cash. This was of headline operating profit. Definition headline operating cash flow stated before cash flow relating to restructuring of and acquisition. Performance bodycote works tirelessly to reduce workplace accidents and is committed to providing a safe. The accident frequency rate has increased to in the year. Further details are included in the corporate responsibility and sustainability section on page. Definition accident frequency is defined as the number of lost time accidents hours approximately man. Performance on a normalised basis the carbon footprint decreased by from tonnes per sales to. Further details are included in the corporate responsibility and sustainability section on page. Definition carbon footprint is defined as tonnes of co equivalent emissions divided by revenue. Co equivalent emissions are calculated by taking electricity and gas usage in kilowatt hours and. Normalised emissions statistics restate prior year figures using current year country specific conversion iea factors. Strategic report governance financial statements additional information stock code boy march pm proof business review. In aircraft engines honeycomb seals are critical to the efﬁciency of gas ﬂow through the. This double role performed by the honeycomb seal results in better fuel efﬁciency and longer. To join the honeycomb seals a form of metal joining called brazing is used. Bodycote has decades of experience providing this complex and specialised service. Bodycote plc annual report for the year ended december march pm proof ade revenue by. Our ade customers tend to think and operate globally and our ade divisions are organised. A large number of bodycote s multinational customers fall within our ade divisions and bodycote. We have facilities around the world including hot isostatic pressing hip and surface technology facilities. The following review reflects constant currency growth rates unless stated otherwise. Revenue in was an increase of at actual rates including a contribution of percentage points. Civil aviation growth was underpinned by a strong uk performance. It was also notable that growth in north american civil aviation revenues picked up through. The revenues from onshore oil gas in north america increased sequentially through the year. These two factors helped the ade divisions achieve revenue growth of in the second half. Headline operating profit was an increase of at actual rates benefiting from positive operational leverage. Accordingly return on sales improved to. Statutory operating profit grew to. Net capital expenditure in was representing times depreciation. In addition to the new hip acquired for our european business our new aerospace facility. Return on capital employed increased to reflecting the improved profitability and careful management of the. . Headline operating profit is reconciled to operating profit in note to the financial statements. Bodycote plants do not exclusively supply services to customers of a given market sector see. Market sector aerospace and defence energy automotive general industrial total geography western europe north america. Because of their function and position within the vehicle they must be extremely strong corrosion. Their effective operation is critical to the safety of the vehicle and therefore the driver. Bodycote s proprietary process ensures the parts achieve the necessary material properties. Bodycote plc annual report for the year ended december march pm proof the agi divisions. Our agi customers include many multinational businesses which tend to operate on a basis as. Much of the business is locally oriented and the business is therefore organised on a. Our extensive network of more than agi facilities enables the business to offer the widest. Bodycote has a long and successful history of servicing this division s customer base. The following review reflects constant currency growth rates unless stated otherwise. Revenue was ahead of the prior year at actual rates including a contribution of percentage. Growth in western european revenues underpinned the divisions growth with growth in its automotive revenues. Emerging markets revenues also grew very strongly and now represent of bodycote s agi business. Mexico and china both achieved revenue growth above. Headline operating profit was ahead of the prior period at actual rates. Return on sales expansion has been a focus for our agi business over many years. Statutory operating profit grew to. Net capital expenditure was representing times depreciation. We are continuing to invest in the rapid growth emerging markets with investments in mexico. We also made further investment in s p as the strong growth in demand for. Return on capital employed increased to reflecting the strong improvement in profitability and is the. . Headline operating profit is reconciled to operating profit in note to the financial statements. Bodycote plants do not exclusively supply services to customers of a given market sector see. Business review automotive general industrial ball studs for further information about our services go to. Because of their function and position within the vehicle they must be extremely strong corrosion. Their effective operation is critical to the safety of the vehicle and therefore the driver. Bodycote s proprietary process ensures the parts achieve the necessary material properties. Agi revenue by market sector and geography market sector aerospace and defence energy automotive general. Acquisitions made in contributed of the constant currency growth with new facilities contributing a further. Headline operating profit for the year increased by to and return on sales increased to. Headline operating profit at constant currency increased by with the five acquired sites in contributing. Price increases more than covered the increase in input costs. Statutory operating profit grew to. Chief financial officer s report dominique y ates chief financial officer bodycote plc annual report. Amounts arising on financial liabilities measured at amortised cost. As at december the group s revolving credit facility is totally undrawn. Having extended the facility during the year it has a remaining life of years. Profit before t axation headline profit before taxation amortisation of intangibles acquisition costs profit before. T ax the passing of the tax cuts and jobs act in the us in. Accordingly the group s tax rate is significantly lower at. The group s headline tax rate for the year excludes this gain and is therefore. The final impact of the changes from the us tax cuts and jobs act are. Bodycote will continue to monitor any developments and give due consideration to the impact of. Earnings per share the improved group business performance drove basic headline earnings per share up. Profit before taxation taxation profit for the year basic headline eps basic eps return on. This improvement was driven by the increase in the group s operating profit. Moreover since the group has invested in growth investment projects many of which are not. The group continues to exert strong financial discipline in the area of capital expenditure as. Cash flow headline operating profit add back items depreciation and amortisation impairment of fixed assets. Earnings before interest tax depreciation amortisation payments impairment of fixed assets profit or loss on. The group s headline operating cash flow increased by to mainly reflecting the improvement in. Statutory net cash from operating activities increased to. Headline operating cash conversion was as the group continues to demonstrate an impressive record of. Consequently free cash flow increased to and the group ended with of net cash. Strategic report governance financial statements additional information stock code boy march pm proof chief financial. The multiple of net capital expenditure to depreciation was times times. The group continues to invest in maintaining its assets to a high quality. More importantly with regard to future revenue growth of the business half of the capital. Acquisitions in december bodycote completed the acquisition of the hip assets and vacuum furnaces from. Deferred consideration payments from acquisitions completed in increased the cash outflow on acquisitions to in. Dividend and dividend policy the group aims to pay ordinary dividends so that dividend cover. The board may also recommend payment of a supplemental distribution to shareholders. The amount of any supplemental distribution will be assessed in light of the cash position. The board has recommended a final ordinary dividend of bringing the total ordinary dividend to. In addition in light of the group s strong balance sheet and year end net. If approved by shareholders both the final ordinary dividend and the special dividend will be. Borrowing facilities the group is financed by a mix of cash flows from operations borrowings. The group s funding policy aims to ensure continuity of finance at reasonable cost based. The group continues to have access to committed facilities at competitive rates and therefore currently. The total undrawn committed facility funding available to the group at december was. At december the group had the following drawings and headroom under the committed facility facility. Alternative performance measures bodycote uses alternative performance measures such as headline operating profit headline earnings. Going concern in determining the basis of preparation for the annual report and the group. This includes an overview of the group s financial position cash flows liquidity position and. The group meets its working capital requirements through a combination of cash resources committed and. The overdrafts and uncommitted facilities are repayable on demand but the committed facilities are due. There is sufficient headroom in the committed facility covenants to assume that these facilities can. The committed facilities as at december were as follows revolving credit facility maturing april the. The group s forecasts and projections taking account of reasonable potential changes in trading performance. The directors have reviewed forecasts and projections for the group s markets and services assessing. The directors also reviewed downside sensitivity analysis over the forecast period thereby taking into account. Following this review the directors have formed a judgement at the time of approving the. For this reason the directors continue to adopt the going concern basis in preparing the. Y ates chief financial officer march bodycote plc annual report for the year ended december. The review of financial risk has been delegated to the audit committee. The group s risk framework using a variety of and approaches is used to identify. The risks are aggregated first at a divisional level and then at group level. For each business critical risk assurance activities have been documented in risk assurance maps and. The group head of risk is supported by the risk and she committee which met. The risk and she committee assists the group head of risk in identifying critical risks. The group head of risk provides an update to the audit committee on the group. The board concluded that an ongoing process of identifying evaluating and managing the group s. The table below highlights the major risks that may affect bodycote s ability to deliver. These risks have been reviewed throughout the year and two new risks have been added. The inclusion of the environment risk reflects increasing regulatory intervention and a level of uncertainty. The inclusion of the capital projects risk reflects the group s continued investments of significant. In determining the principal risks the board once again considered the result of the referendum. The board does not expect this will have a material impact on bodycote as customers. Details of the group s financial risks funding foreign exchange interest rate and counterparty risks. The mitigating activities described below will help to reduce the impact or likelihood of the. The board recognises that there could be risks that may be unknown or that may. For this reason business continuity plans have been prepared for all plants to provide for. Risk description impact mitigation and control relevance to strategy market and customer risks markets bodycote. The result of the referendum on the future of the uk s membership in the. Stable the high proportion of fixed costs in the business means that a drop in. Sales in the markets served by the agi businesses of the total group tend to. Sales to the energy sectors are closely linked to energy prices which in turn can. Bodycote s presence in countries across a wide variety of acts as a natural hedge. There is some flexibility in the cost base. By ensuring that a proportion of the workforce is employed on temporary contracts. Changes in customer demand on a local or a level are responded to quickly. Loss of key customers bodycote benefits from many relationships with key customers and the damage. Stable the loss of a key customer could adversely affect the group s financial results. There is no significant customer dependency with the group s top ten customers accounting for. There is a continued focus on customer service and quality processes to maintain excellent relationships. Key account management is in place and customer satisfaction is monitored. Strategic report governance financial statements additional information stock code boy march pm proof principal risks. Stable the erosion of market share resulting in loss of revenue and profit. The close control of proprietary knowledge. Rapid increase in the scale of the group s offerings to maintain the position as. Corporate and community risks safety and health the nature of bodycote s activities presents safety. Stable bodycote is committed to providing a safe work environment for its employees but bodycote. Furthermore poor safety and health practices could lead to disruption of business financial penalties and. Health and safety policies set by the group chief executive. Ohsas and iso compliant she management systems being used by group head of safety health. Programme in place to focus on reduction of incidents which could have a high impact. Safety compliance audits at all plants at least every two years. Oversight of safety and health framework provided by the group risk and she committee. Environment actual or potential environmental contamination could lead to health risks disruption of business financial. Increasing bodycote is committed to providing the highest level of protection to the environment. Environmental regulators in many jurisdictions in which bodycote operate can impose obligations on bodycote to. Environmental procedures and measures in place conforming to iso of plants. Environmental due diligence of businesses for acquisition. Remediation of contaminated sites or additional emission abatement as required by local legislation. Bodycote plc annual report for the year ended december march pm proof risk description impact. Increasing deterioration in quality or service levels can cause serious damage to bodycote s reputation. Work that is released into use which is not in compliance with specification could arise. The automotive iso technical specification has recently undergone major revision and is being replaced by. The new standard requires additional work for example additional quality inspections. Bodycote has stringent quality systems in place managed by qualified staff. Quality systems and processes operated at plant level with oversight by divisional quality teams. Where necessary plants maintain industry relevant accreditations such as iso nadcap and iatf. All plants subjected to internal and external quality audits and inspections at least once a. Major disruption at a facility bodycote s business processes are inherently risky and there is. In addition a number of sites are exposed to natural hazards such as earthquakes flooding. Stable any significant incident at a site could result in the service to bodycote s. Bodycote has a global network of facilities. These facilities create a framework to provide capability for affected facilities. Business continuity plans are in place for all plants. These are updated and tested annually. This process has been subject to a board risk deep dive in. Independent insurer inspections to assess hazard and business interruption risks. Insurance cover including business interruption cover. Scheduled equipment maintenance and inspections. Capital projects the group invests capital in developing existing plants as well as into greenfield. Increasing capital projects can be highly complex and rely on factors outside of the group. This may cause projects to be delivered late or at a higher cost than forecast. Market conditions may also change making a project less profitable than initially projected. There is a well established capital investment approval process that applies to all major capital. Project management frameworks are in place to deliver projects on time and on cost. All major projects are subject to post implementation reviews. Capital project risk will be subject to internal audit review in. Strategic report governance financial statements additional information stock code boy march pm proof principal risks. Bodycote is currently undergoing a group wide implementation of an erp system. Stable failure to protect the group s it systems from cyber threats or to maintain. Project approval and progress subject to regular executive committee and board review. Project teams made up of skilled subject matter experts supplemented with third party advisers. Best practice project management processes in place with assurance provided by third parties. Defined disaster recovery planning and data backup procedures. Regulatory risks regulatory and legislative compliance the global nature of bodycote s operations means that. Stable failure to comply with legislation could lead to substantial financial penalties disruption to business. Business processes are supported by hr policies and the group code of conduct alongside training. The open door line blower facility which is managed by a third party. Engagement of local specialists to support bodycote at local divisional and group level. Regular audit of the effectiveness of implemented procedures. Bodycote plc annual report for the year ended december march pm proof viability statement in. This longer term assessment process supports the board s statements on both viability as set. A three year period was determined as it is a reasonable period over which the. As a result the board determined that a period of longer than three years would. The forecast used considers metrics which enable assessment of the group s key performance indicators. In conducting the review of the group s prospects the directors assessed the three year. This assessment considered the impact of the principal risks on the business model and on. The directors viability assessment included a review of the sensitivity analysis performed on the three. The developed scenarios were designed to be plausible yet severe. Examples of scenarios reviewed were a decrease in forecast revenue of similar magnitude to the. A decrease in revenue debtor days and sterling strengthening to reflect an economic downturn in. The directors also took into consideration the group s financial position at december with net. The directors have assessed the viability of the group and based on the procedures outlined. Strategic report governance financial statements additional information stock code boy march pm proof corporate responsibility. Bodycote plc annual report for the year ended december march pm proof bodycote s stakeholder. These exchanges based on mutually beneficial relationships provide the basis for the group s growth. Taxes productivity return on investment services remuneration payment bodycote provides thermal processing services that improve. . . Improves the lifetime and performance of products supports businesses and protects lives investors funders capital. Sales services employees employees knowledge expertise and skill are a major part of the group. Was paid out as remuneration. Customers our services are provided aerospace defence energy and general industrial industries. Suppliers suppliers proﬁt from the location of the group in local communities and from the. Society bodycote generates wealth for society and contributes to socioeconomic development through its sustainable business. In total employer related social taxes net vat corporate and other transnational taxes amount to. We aim to achieve this in a safe working environment while continually seeking to minimise. Bodycote is dedicated to improving the management of corporate responsibility issues and is implementing policies. The future success and growth of the group is intrinsically linked to our ability to. Our people the strength of the group primarily rests in its people and one of. Bodycote is fortunate to have a competent and committed international team that is respected in. Bodycote invests in the training and development of its people both at local and group. At a local level the group is committed to providing the appropriate skills and technical. At group level a number of initiatives are currently being rolled out to drive excellence. A tool to develop further understanding and skill in the area of performance management is. Through communication of clear messages coupled with skills development the organisation aims to raise the. This initiative is backed by a performance management system which supports the process. Bodycote s employment policies are complying with all current legislation to engender equal opportunity irrespective. Harassment is not tolerated. Female representation on our board during was and at manager level it is. Females represent of our total workforce. As of january we have increased female representation on the board to. Male female t otal male female t otal directors managers other staff. Accident frequency is defined as the number of lost time accidents hours approximately man years. . Note the rate for has increased from as previously reported last year to due to. . Co e is carbon dioxide equivalent which represents the co release due to our energy. . Normalised statistics restate prior year figures using current year iea carbon conversion factors and current. Bodycote plc annual report for the year ended december march pm proof it is not. How we operate as a group and the behaviours that we expect from all our. Our values represent bodycote and its people and our commitment to the company and the. Our core values are straightforward and are as follows honesty and t ransparency we are. Trust stems from honesty and trust is at the heart of everything we engage in. This is not something we take for granted. Bodycote lives by a culture of honest and transparent behaviour which is at the core. Respect and responsibility we manage our business with respect applying an ethical approach to our. We respect our colleagues who are all of the employees of bodycote. Part of our respect for our colleagues is our commitment to safe and responsible behaviour. We show respect for our customers our suppliers and our competitors. We respect the communities around us and behave as responsible corporate citizens by being compliant. We believe in taking ownership for and being mindful of the impact of our actions. Creating v alue creating value is the very essence of our business and needs to. We create value for our customers our employees and our shareholders. The realities are harsh. If we do not create value for our customers then we have no reason for. If we do not create value for our employees there will be to create value. Our shareholders rightfully require that we ultimately create value for them as they are the. Human rights bodycote s human rights policy is consistent with the universal declaration of human. We prohibit forced compulsory and underage labour and any form of discrimination based on age. Appropriate mechanisms are in place to minimise the potential for any contravention of these rules. By publicly posting our human rights policy on stakeholders worldwide can alert us to potential. Our internal systems also support compliance with our policy and we have a robust open. In the jurisdictions in which we employ a majority of our employees there are laws. The modern slavery act bodycote plc has conducted a risk assessment on our supply chain. Suppliers in those countries identified in walk free foundation s global slavery index as being. We have a code of conduct which sets out our policy on compliance with legislation. The and human trafficking statement was reviewed by our board of directors in september and. The statement will be reviewed on an annual basis. Customers and suppliers bodycote has no significant suppliers who are wholly dependent upon the group. Suppliers are paid in line with contractual and legal obligations. We endeavour to respond quickly to changing customer demand to identify emerging needs and to. We stay close to our current and potential customers building relationships. Community bodycote seeks to play a positive role in the local communities in which it. Responsible business ethics all bodycote personnel are expected to apply a high ethical standard consistent. Directors and employees are expected to ensure that their personal interests do not at any. Shareholder employees are advised of and comply with the share dealing code. Bodycote has systems in place that are designed to ensure compliance with all applicable laws. Furthermore bodycote does not make political donations. With regard to competition bodycote aims to win business in a differentiated manner. The group does not employ unfair trading methods and it competes vigorously but fairly within. Employees are prohibited from either giving or receiving any inducements. Our open door policy has been translated into all languages used throughout the group. The policy allows employees to report their concern confidentially verbally or in writing to an. Reports are transcribed and sent to the group head of risk who then determines the. Online training courses in respect of and competition law have been designed and translated into. All relevant employees have completed the interactive courses. Strategic report governance financial statements additional information stock code boy march pm proof corporate responsibility. We are committed to complying with all local legislative requirements as a minimum and establishing. Safety and health the nature of the group s operations is such that employees are. Bodycote aims to manage these hazards and thereby minimise risks to employees through the deployment. Bodycote s online incident reporting and she management tool has been operational since. This has enabled more consistent and thorough reporting of workplace injuries near misses and unsafe. Following the implementation there was an increase in the lost time injury rate frequency lti. In the lti rate increased from to. Bodycote continues to devote significant resource and management focus on safety with the number of. This improvement demonstrates stronger engagement of employees in proactively raising safety issues and rectifying potential. Accidents though regrettable and unacceptable represent learning opportunities. This is the reason that accurate reporting is an essential part of building a robust. Accident frequency lost time injury rate accident frequency is defined as the number of lost. In addition to encouraging the reporting of work related injuries bodycote has sought to encourage. This has worked well since the introduction of the new global incident reporting system in. This much improved reporting of incidents permits us to address hazards before injury occurs. As our database continues to develop we will be able to analyse and prioritise our. The most frequent cause of lost time incidents is related to manual handling of parts. This is currently the subject of a group wide review and will be a focus. All reportable incidents and lost time injuries are reviewed during executive management meetings and board. In addition the executive management team reviews incidents which did not result in injury but. All serious incidents and high potential incidents are also reviewed by the group she committee. This system was further strengthened in with actions being tracked via the online incident management. Environment a proactive approach to improving energy efficiency means that bodycote has implemented a variety. The ongoing effort to lessen the impact on the environment has resulted in bodycote seeking. At every stage where bodycote is involved in the manufacturing cycle our operational aim is. Bodycote operates modern efficient equipment which is operated around the clock so as to optimise. Without bodycote many companies would be using older technology and running their equipment at reduced. Working with bodycote enables our customers to commit more easily to carbon reduction initiatives. Bodycote also reduces the carbon footprint of our customers activities by increasing the lifespan of. For example surface treatment technology is widely used in the reclamation of damaged and worn. The treated parts often last up to twenty times longer than the original. Whilst thermal processing is an business it is a vital part of the manufacturing supply. Greenhouse gas emissions co e emissions ktco e intensity ratio tco co e emissions ktco. Normalised statistics restate prior year emissions using current year iea carbon conversion factors and current. Emissions per of turnover. Bodycote plc annual report for the year ended december march pm proof scope emissions are. Scope emissions are indirect energy emissions resulting from purchased electricity heat steam or cooling for. The financial control consolidation approach has been used to report the above data. This method aligns with the reporting scope in the financial statements. The group collects electricity and natural gas usage information from each facility on a monthly. The group then applies the international energy agency iea published national carbon conversion factors to. Group operational management actively monitors their monthly co e emissions reported and the group s. All entities and facilities under financial control are included within the disclosure. Emissions less than of the group s total co e relating to fugitive emissions and. As such there are no significant omissions from this disclosure. Iso accredited facilities reducing the environmental impact of the group s activities is taken very. Compliance with the requirements of iso helps to minimise the risk of adverse environmental effects. At the end of of our operating facilities had achieved iso accreditation. The slight reduction is due to the closure of a number of existing certified sites. Operational plants which have not yet received accreditation to the standard are working towards it. Carbon footprint and water consumption the absolute energy usage increased by. At constant exchange rates it increased by. The total co e emissions per sales in were te as previously reported te te. The group s total co e emission data is based on scope and scope emissions. In previous years this has been supplied by defra directly. However as of january defra no longer supplies these conversion factors for companies. This has now been sourced by the group directly from the international energy agency iea. There are some significant differences in these conversion factors. As a result all previous years have now been restated using iea conversion factors to. On a basis water usage per sales decreased by. On a basis water usage per showed a decrease of. In our eu based operational sites reviewed their operations to ensure compliance with the energy. This directive is transposed into local legislation and requires sites to monitor their energy usage. One mechanism for ensuring compliance is for sites to become certified to iso energy management. This enables sites to measure energy usage consistently and target the most effective ways of. Our sites in germany austria denmark and the netherlands are largely already certified and working. Bodycote uses established systems to develop best practice at specific sites and across the wider. Examples of projects undertaken across bodycote sites are discussed below. The continued replacement of traditional lighting with led for environmental and improved safety has resulted. Our sites at sprockhövel otterfing esslingen korntal wehingen gothenburg and warsaw will benefit from projected. At the lüdenscheid plant the second phase of a heat recovery project will save a. At langenfeld co savings will be achieved from new furnace insulation. A new heat exchange system in gothenburg for both cooling water and process ventilation has. Meanwhile in denmark our ejby plant will reduce co emissions by through the new air. These replace evaporative water cooling towers and in addition to the energy savings eliminate the. In addition to process efficiency improvements some sites have upgraded the building fabric to improve. At our haag site a project to replace windows and improve insulation resulted in a. Since bodycote has submitted data on co usage to the carbon disclosure project one of. Each year the company has improved its standing in the league tables and is now. Chlorinated solvent use the use of chlorinated solvents in bodycote s thermal processing activities has. In the solvent use showed a further decrease of compared with the previous year. Cautionary statement the strategic report has been prepared solely to provide information to shareholders to. The strategic report contains certain statements. These statements are made by the directors in good faith based on the information available. Approval the group strategic report of bodycote plc was approved by the board of directors. Strategic report governance financial statements additional information stock code boy march pm proof board of. Director and chairman of the audit committee of babcock international group plc since and a. External roles external roles past roles past roles worked on a variety of audits with. From to he worked for lloyds bank plc and then switched to british nuclear fuels. In he took on the role of group finance director with royal mail holdings plc. He was director of fiberweb plc during mouchel group from to and wandisco plc from. Spent his early career in engineering with courtaulds plc and then moved to the usa. He was appointed to the board of powell duffryn plc as an executive director in. He was also a executive director of brixton plc from to. Qualiﬁcations qualiﬁcations chartered engineer graduated from cambridge university masters degree in business administration from the. Dominique y ates chief financial officer appointed november ian duncan senior independent director appointed november. Worked in various roles for nz forest products ltd followed by management consultancy with resource. Managing director of riverstone holdings llc from. Director of boc group plc from to director and remuneration committee chair as well as. University of auckland and msc management sciences massachusetts institute of technology. International operations emerging markets mergers and acquisitions management leadership manufacturing capital intensive industry managing director. Held various senior positions in imperial tobacco group plc followed by chief financial ofﬁcer positions. Chartered accountant graduated from bristol university in economics and accounting. Leadership international operations mergers and acquisitions emerging markets current ﬁnancial experience service industry chartered accountant. International operations current ﬁnancial experience supply chain and logistics mergers and acquisitions service industry executive. Began her career in various positions with ericsson working in continental europe north america and. Joined teliasonera in as senior vice president moving to xelerated initially as chairperson and later. Director of transmode holdings ab from to blekinge institute of technology from to tieto corporation. Eva will retire from caverion oy in march from assa abloy ab in april and. Engineer graduated with a masters from linköping institute of technology diploma in marketing from ihm. Past roles qualiﬁcations skills and experience qualiﬁcations pat larmon director appointed september ute ball group. Lili began her career as an actuary before joining bain company. Graduated with a bsc in mathematics from concordia university montreal followed by an mba from. Associate of the society of actuaries. Strategy and consultancy international operations mergers and acquisitions oil gas industry business services industry oil. Chief executive ofﬁcer north america of bunzl plc since joining the bunzl plc board in. Was executive vice president and owner of packaging products corporation until when the company was. Held various senior management positions for over years before becoming president of bunzl s north. Graduated from illinois benedictine university major economics business economics followed by achieving certiﬁed public accountant. International operations mergers and acquisitions service industry manufacturing distribution sales and marketing chief executive ofﬁcer. Key to committees audit remuneration nomination executive committee chair e n r a n r. The corporate governance statement provides an insight into how the board operated during the year. We are committed to conducting business responsibly. By maintaining high standards of corporate governance we enhance performance underpinned by our business model. Our approach to governance is set by the board and our executive committee ensures that. Effective and robust governance remains central to the ongoing success of the group. The main governance documents are our core values and the code of conduct which set. These documents together with our policies govern how we conduct our business and set the. Compliance training helps to enforce this. Board oversight reviews and audits form part of the monitoring and supervision process. Risk processes are embedded and reviewed on an ongoing basis across the business. The important governance developments at bodycote over the last year are detailed in the governance. My ambitions for the composition of the board are to maintain and where applicable broaden. The board continues to ensure that effective succession plans are in place. I encourage all shareholders to attend the agm which will be held at our macclesfield. This event provides an excellent opportunity to meet the executive and independent directors. Quinn chairman board performance key actions achievements priorities for implement actions from the strategy review. The nomination committee considers diversity when making appointments to the board taking into account relevant. Our prime responsibility however is the strength of the board and our overriding aim in. The nominations committee also considers capability and capacity to commit the necessary time to the. The intention is to appoint the most suitable qualified candidate to complement and balance the. The nomination committee is advised by international search companies who have been briefed on our. We appointed quinn on january as part of our board refreshment replacing thomson who retired. Chahbazi was appointed director effective january. The board currently comprises two executive directors four directors and a chairman. As of female representation on our board is. At manager level it is. Females represent of our total workforce. Whilst we are above the by voluntary target recommended by the review we continue to. We will increase female other minority representation on the board if appropriate candidates are available. The corporate responsibility and sustainability report contains further details regarding the male and female representation. Board evaluation following the external board evaluation in the board agreed to undertake an internal. To ensure that all aspects of good governance are covered by the review the group. Questions were framed under the following seven topics remit and objectives composition training and resources. At a meeting of the nomination committee in september the directors assessed the conclusions reached. Additional emphasis will be placed on risk management strategy and operational matters. The board evaluation covered the activities of the main board and each of its committees. The board is considered to be functional and working well. Arising from the exercise the board concluded that its focus should remain on divisional growth. The overall conclusion is that the board is performing well and high governance standards have. The executive committee is strongly challenged by the board when appropriate. As in previous years the chairman has assessed the performance of each board member by. The executive directors messrs harris and will be appraised in march. Led by the senior independent director the directors carried out an evaluation of the chairman. The board was satisfied with the now retired chairman s commitment and performance. Overboarding at our agm in bodycote received a high number of votes against the of. Eva has now had time to review her directorships and will not stand for at. Rns announcements will be made during confirming that eva will not stand for at the. Following these changes eva lindqvist will sit on five listed boards bodycote another uk plc. Following a review of eva s time commitments and taking into account the reduced number. Financial statements additional information stock code boy strategic report governance march pm proof corporate governance. They also meet all other directors and senior executives. This facilitates their understanding of the group and the key drivers of business performance. T raining the board receives training via ad hoc presentations and papers from advisers and. External periodic training on important topics takes place and during the year the directors received. Other opportunities for development and support are a programme of visits throughout the year reviews. Succession planning succession planning ensures that appropriate senior executive leadership resources are in place to. The plans are reviewed annually by the nomination committee. The board further develops its knowledge and gains greater visibility of executive talent and management. Tenure in years patrick larmon lili chahbazi anne quinn tenure in years eva lindqvist ian. This is monitored by the board at regular intervals. Further details are available on page. Rights in line with best practice provisions in the group statement of principles the board. Financial statements additional information stock code boy strategic report governance march pm proof corporate governance. In respect of the year ended december bodycote has complied with the provisions of the. Regarding the board has in recent years taken the view that generally it is the. The chairman also meets institutional investors to discuss overall strategy governance and any concerns that. Only where these more usual channels of communication have failed would the board expect the. The sid has contacted major shareholders and offered to facilitate meetings with them should they. Regular feedback from the group s advisers on investor meetings and results presentations is circulated. During the year the chairman met with shareholders to discuss governance matters. Apart from these distinct areas bodycote was in compliance with the provisions of the code. Operation of the code taken together with the report of the audit committee the report. Leadership the board is responsible to shareholders for good corporate governance setting the group s. The board met on nine occasions during including a specific meeting to review and update. The board of directors comprises seven members of whom five are directors and two are. The group chief executive is harris and the senior independent director is. Duncan who also chairs the audit committee. Lindqvist is chair of the remuneration committee. P. Larmon and chahbazi are directors. Brief biographical details of all directors are given on pages to. During the year the board visited a number of uk and overseas facilities including sites. Such events involved meetings with local management and the unit workforce to understand more clearly. Matters reserved for the board were reviewed during the year and updated where required. Certain defined powers and issues reserved for the board to decide are inter alia strategy. Bodycote plc annual report for the year ended december march pm proof in advance of. They are also supplied with the latest available information on safety health and environmental and. Senior management from across the group and advisers attend some of the meetings to provide. The exposure to members of senior management from across the group helps enhance the board. Where required a director may seek independent professional advice the cost of which is reimbursed. All directors have access to the group company secretary and they may also address specific. In accordance with the articles of association all newly appointed directors must submit themselves for. All directors stand for yearly. Directors including the chairman are appointed for fixed terms not exceeding three years from the. A statement of the directors responsibilities is set out on page. The board also operates three committees. These are the nomination committee the remuneration committee and the audit committee. All directors serve on each board committee. In accordance with the recommendations of the code board members serve for a period of. If letters of appointment are extended beyond six years the fixed term is reduced to. In order that necessary actions can be taken promptly a finance comprising the chairman or. Shareholders audit committee monitors the integrity and effectiveness of the group s nancial reporting and. Larmon lindqvist. Duncan and chahbazi are all independent for the purposes of the code. The chairman was considered independent upon appointment. Commitment attendance of directors at regular scheduled meetings of the board and its committees is. Duncan p. Larmon y ates. Thomson resigned from the remuneration committee on july. . Harris resigned from the nomination committee on july. All directors attended the maximum number of board audit and nomination committee meetings that they. P. Larmon did not attend one remuneration committee due to an unpredictable diary clash. In addition messrs thomson harris attended by invitation some parts of the meetings of the. Proposals for the board decided in line with the code that all directors will retire. Accordingly harris lindqvist p. Larmon. Duncan and yates will stand for at the agm in may. Having been appointed since the last agm quinn and chahbazi will stand for election. The board recommends to shareholders that they or elect all the directors. In accordance with the recommendations of the code board members will serve for a period. The performance of each director was evaluated as indicated above and the board confirms in. Internal control and risk management the board is responsible for the group s system of. Such a system is designed to manage rather than eliminate the risk of failure to. The board has applied principle of the code by establishing a continuous process for identifying. The board s monitoring covers all controls including financial operational and compliance controls and risk. It is based principally on reviewing reports from management and from internal audit ia to. The audit committee assists the board in discharging these review responsibilities. Bodycote plc annual report for the year ended december march pm proof the board believes. The system was in operation throughout and continues to operate up to the date of. Key elements of the group s system of internal control are as follows the group. The group s authority matrix was reviewed and updated during and this clearly sets out. The board with the assistance of ey who provide ia services monitors the group s. Ia reviews are conducted on the basis of a risk based plan approved annually by. This includes regular visits to each division shared service centres and plants. The findings and recommendations from ia are reported on a regular basis to the executive. An annual internal control with management certification is undertaken by every bodycote site. The assessment covers the effectiveness of key financial and compliance controls and was revised at. The results are validated by ia through spot checks and are reported to the executive. Group core values and group policies including the code of conduct group authority matrix and. The chief financial officer group financial controller president and vice president of finance for each. This is to confirm the adequacy of their systems of internal controls their compliance with. A risk register and assurance map is maintained throughout the year to identify the group. Any changes to these risks during the year are promptly reported to the executive committee. During in compliance with provision of the code management performed a specific assessment of its. Management s assessment which has been reviewed by the audit committee and the board included. The review was based on work performed by the group head of risk and the. These risks have been reviewed throughout the year and two new risks have been added. Further information regarding the ways in which the principal business risks and uncertainties affecting the. Investor relations the group chief executive and chief financial officer regularly talk with and meet. In addition internet users are able to view news on the group and its share. Users of the website can access recent announcements and copies of results presentations and can. On a regular basis bodycote s financial advisers corporate brokers and financial public relations consultants. The chairman and sid are available to discuss any issues not resolved by the group. On specific issues such as the review of remuneration packages the group has sought and. By order of the board ball group company secretary march springwood court springwood close tytherington. The chairman s statement the chief executive s review the chief financial officer s report. Strategic report the strategic report is provided on pages to of this annual report. This is a review of the development of the group s businesses the financial performance. The strategic report has been prepared solely to assist the shareholders in assessing the group. It should not be relied on by any other party for any other purpose. Statements have been made by the directors in good faith using information available up to. Since the end of the financial year no important events affecting the business of the. Dividends the board has recommended a final dividend of bringing the total ordinary dividend to. If approved by shareholders the final dividend of per share will be paid on june. A special dividend of has also been proposed and will be subject to shareholder approval. If approved it will be paid on june. Share capital the company s issued ordinary share capital as at december was. No shares were issued during the year. At the agm on may the shareholders authorised the company to purchase up to of. This authority expires at the conclusion of the forthcoming agm to be held on may. Capital structure details of the issued share capital are shown in note. The company has one class of ordinary shares which carries no right to fixed income. Each share carries the right to one vote at general meetings of the company. There are no specific restrictions on the size of a holding nor on the transfer. The directors are not aware of any agreements between holders of the company s shares. Details of employee share schemes are set out in note and shares held by the. No person has any special rights of control over the company s share capital and. The appointment and replacement of directors is governed by the company s articles of association. The articles of association may be amended by a special resolution of shareholders. The powers of the directors are described in the corporate governance statement on page. Under the articles of association the company has authority to issue ordinary shares with a. There are also a number of other agreements that take effect alter crystallise or terminate. None of these are considered to be significant in terms of their likely impact on. Directors the current directors and their biographical details are listed on page to and all. Thomson retired as chairman and director as of december and was replaced by quinn as. A further executive director chahbazi was appointed effective january. Under the articles of association of the company each director must retire from office and. However in line with the uk corporate governance code and to further increase accountability all. Going forward all directors will retire at the agm and will stand for by the. Accordingly those directors retiring and offering themselves for at the agm are harris. Duncan lindqvist p. Larmon and yates. Since quinn and chahbazi were appointed after the agm in both will stand for election. The service agreements for messrs harris and yates are terminable by months notice. The remaining directors do not have a service agreement with the company and their appointments. Directors interests in contracts and shares details of the executive directors service contracts and details. No director has had any dealings in any shares or options in the company since. Qualifying third party indemnity provision as defined by section of the companies act has remained. Apart from these exceptions none of the directors had a material interest in any contract. Bodycote plc annual report for the year ended december march pm proof potential conflicts of. Potential conflicts of interest could arise where a single director owes a fiduciary duty to. In order to ensure that each director was complying with the duties each director provided. The board reviewed the declarations and approved the existence of each declared situational conflict up. Where such authorisation becomes inappropriate or not in the interests of bodycote shareholders the chairman. No such revocations have been made. Employment the group recognises the value that can be added to its future profitability and. The commitment of employees to excel is key to the group s continued success. Through their attendance at or participation in strategy production safety and health meetings at site. Under the group s open door line employees concerns can be voiced over the phone. Approximately bodycote employees are connected to the bodycote intranet which improves knowledge of group activities. It is the group s policy to give full and fair consideration to applications for. Should an employee become disabled the group where practicable will seek to continue the employment. An equal opportunities policy is in operation in the group. Greenhouse gas emissions details of greenhouse gas emissions are included within the corporate responsibility and. Donations there were no political contributions in or. Shareholders an analysis of the company s shareholders and the shares in issue at february. Auditor in accordance with the provisions of section of the companies act a resolution for. Each person who is a director at the date of approval of this annual report. This statement is given and should be interpreted in accordance with the provisions of section. Annual general meeting the annual general meeting will be held on may in accordance with. By order of the board ball group company secretary march springwood court springwood close tytherington. Of meetings main committee responsibilities director attendance regularly review the structure size and composition including. Give full consideration to succession planning for directors and other senior executives in the course. Be responsible for identifying and nominating for the approval of the board candidates to fill. Thomson harris. Duncan lindqvist p. Larmon resigned membership on july dear shareholders i am pleased to introduce the nomination committee. The committee s key objective is to support the board in fulfilling its responsibilities to. This was a busy year for the committee. In addition to its main responsibilities the committee oversaw the process of appointing a new. Alan thomson chairman retired on december after more than nine years of service. Lili chahbazi was appointed director on january at the same time as i took over. The committee will continue to focus on ensuring that the present and future composition of. Quinn chairman of the nomination committee role of the nomination committee the nomination committee is. The committee s terms of reference which are listed on the group s website include. Further information on the code can be found on the financial reporting council s website. . The terms of reference are reviewed annually by the group company secretary and the chairman. No changes were made to the terms of reference during the year. Composition of the nomination committee as recommended by the code the chairman of the board. The chairman can not chair the committee when it is dealing with either the succession. Only members of the committee have the right to attend the committee meetings. Other individuals and external advisers may be invited to attend for all or part of. The quorum necessary for the transaction of business is two. The group company secretary is secretary to the committee. The committee has the authority to seek any information that is required from any officer. In connection with its duties the committee is authorised by the board to take such. Board composition recruitment selection interview balance of skills appointment induction vacancy for a director is. The need for specific knowledge skills and role behaviours is identified during discussions at nomination. External international search consultancies were appointed to assist with the search. Zygos assisted the sid in the chairman s recruitment and russell reynolds assisted the chairman. A examined the long list of candidates against the role specifications and a shortlist of. Candidates were initially interviewed by the sid and the chief executive for the chairman s. The final candidates then met with all other directors. In order to maximise the effectiveness of the board an additional director was appointed increasing. Anne quinn and lili chahbazi were announced as joining the board. Anne succeeds alan thomson as chairman. The committee and the group company secretary will play an active part in an induction. Succession planning bodycote plc annual report for the year ended december march pm proof director. Prior to making any appointment the committee having evaluated the skills experience and diversity of. The committee will use open advertising or the services of independent external advisers to facilitate. A long list of candidates will be drawn up from which an appropriate number will. Upon completion the committee will recommend to the board the appointment of the preferred candidate. Board succession planning upon the retirement of thomson on december he was replaced by quinn. The recruitment process was led by the senior independent director. Duncan. Ian was advised by international search consultancy zygos in the process of identifying suitably qualified. Lili chahbazi joined the board as a director on january and thomson was advised by. Neither zygos nor russell reynolds have other connections to bodycote plc. Main activities of the nomination committee in the committee formally met seven times and reviewed. The committee discussed succession planning and board diversity and reviewed the performance of the group. In particular the need to broaden the board membership with respect to gender ethnicity and. The committee has sought to ensure that appointments are of the best candidates to promote. Further information concerning board diversity can be found on page as part of the corporate. We are pleased to report that as of january the female representation on the board. The committee considered and authorised the potential conflicts of interest which might arise where a. The committee concluded that no inappropriate conflicts of interest exist. The committee also assigned the chairman to review and agree with the group chief executive. Following the external board evaluation in the board agreed to undertake an internal evaluation during. Further details of the review can be found in the corporate governance section of the. Recommendations arising from the board evaluation are in the process of being addressed. In our agm results announcement we stated that eva lindqvist had confirmed to reduce her. Consequently eva will not stand for at three listed companies caverion oy in march assa. This will reduce eva s other directorships to five. In our view this enables eva to allocate sufficient time to bodycote to discharge her. In december the nomination committee reviewed the board s size and composition the frequency of. The terms of reference of the committee were reviewed in conjunction with the model terms. The biographical details of the current directors can be found on pages and. The committee having reviewed their independence and contribution to board matters confirms that the performance. Accordingly the committee has recommended to the board that all current directors of the company. As chairman of the committee i will be available at the agm in may to. On behalf of the nomination committee quinn cbe chairman of the nomination committee march nomination. Of meetings main committee responsibilities director attendance encourage and safeguard the highest standards of integrity. Monitor the integrity of the financial statements including annual and reports trading updates and any. Reviewing and reporting to the board on significant financial reporting issues and judgements. Review the content of the annual report and advise the board whether taken as a. Monitor and review the adequacy and effectiveness of the company s internal financial control and. Oversee the relationship with the external auditor including consideration of fees audit scope terms of. Monitor and review the effectiveness of the company s internal audit function. Review the adequacy and security of the company s arrangements for its employees to raise. . Duncan lindqvist p. Larmon introduction the committee has continued to focus on the integrity of bodycote s financial. The committee will continue to keep our activities under review as the regulatory environment changes. Membership the members of the audit committee are all independent directors. Their biographical details are shown on pages to and their remuneration on page. .

# ***Genearating  Summary of all files***

In [10]:
import os

# Path to the annual_reports folder
reports_folder = '/content/drive/MyDrive/validation (2)/annual_reports (1)'

# Path to the NEW SUMMARYYYYYY folder
summary_folder = '/content/drive/MyDrive/updated GENERATED SUMMARY'

# Create the summary folder if it doesn't exist
os.makedirs(summary_folder, exist_ok=True)

# Function to correct full stops and capitalize sentences
def correct_full_stops_and_capitalize(summary):
    # Replace all occurrences of double full stops with single full stops
    corrected_summary = summary.replace("..", ".")

    # Capitalize the first letter of each sentence
    sentences = corrected_summary.split(". ")
    corrected_sentences = [sentence.strip().capitalize() for sentence in sentences]

    # Join the corrected sentences back together
    corrected_summary = ". ".join(corrected_sentences)

    # Ensure the last character is a full stop
    if not corrected_summary.endswith("."):
        corrected_summary += "."

    return corrected_summary

# Iterate through files in the annual_reports folder
for filename in os.listdir(reports_folder):
    if filename.endswith('.txt'):
        # Read the content of the file
        file_path = os.path.join(reports_folder, filename)
        document = read_text_file(file_path)

        # Preprocess text
        processed_text = preprocess_text(document)

        # Topic modeling with LDA
        num_topics = 20
        lda_model, topics, corpus, dictionary, coherence_lda = lda_topic_modeling(processed_text, num_topics=num_topics)

        # Extractive summarization using TextRank
        summary = text_rank_summarization(processed_text)

        # Correct full stops and capitalize sentences
        corrected_summary = correct_full_stops_and_capitalize(summary)

        # Save summary to a file in the summary folder
        summary_file_path = os.path.join(summary_folder, f"{filename.split('.')[0]}.txt")
        with open(summary_file_path, 'w', encoding='utf-8') as file:
            file.write(corrected_summary)

        print(f"Summary saved successfully in {summary_file_path}")


Summary saved successfully in /content/drive/MyDrive/updated GENERATED SUMMARY/30777.txt


Summary saved successfully in /content/drive/MyDrive/updated GENERATED SUMMARY/30778.txt


Summary saved successfully in /content/drive/MyDrive/updated GENERATED SUMMARY/30781.txt


Summary saved successfully in /content/drive/MyDrive/updated GENERATED SUMMARY/30779.txt


Summary saved successfully in /content/drive/MyDrive/updated GENERATED SUMMARY/30782.txt


...

Summary saved successfully in /content/drive/MyDrive/updated GENERATED SUMMARY/33130.txt


Summary saved successfully in /content/drive/MyDrive/updated GENERATED SUMMARY/33097.txt


Summary saved successfully in /content/drive/MyDrive/updated GENERATED SUMMARY/33098.txt


Summary saved successfully in /content/drive/MyDrive/updated GENERATED SUMMARY/33154.txt


Summary saved successfully in /content/drive/MyDrive/updated GENERATED SUMMARY/33155.txt


# **Renaming Genearted summary**

In [11]:
import os

# Path to the directory containing the files
folder_path = "/content/drive/MyDrive/updated GENERATED SUMMARY"

# Iterate over all files in the directory
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # Check if the file is a text file
        # Extract the file number from the filename
        file_number = filename.split('.')[0]

        # Define the new filename pattern
        new_filename = f"fns{file_number}_{filename}"

        # Build the full paths for the old and new filenames
        old_filepath = os.path.join(folder_path, filename)
        new_filepath = os.path.join(folder_path, new_filename)

        # Rename the file
        os.rename(old_filepath, new_filepath)
        print(f"Renamed '{filename}' to '{new_filename}'")

Renamed '30777.txt' to 'fns30777_30777.txt'
Renamed '30778.txt' to 'fns30778_30778.txt'
Renamed '30781.txt' to 'fns30781_30781.txt'
Renamed '30779.txt' to 'fns30779_30779.txt'
Renamed '30782.txt' to 'fns30782_30782.txt'
Renamed '30783.txt' to 'fns30783_30783.txt'
Renamed '30785.txt' to 'fns30785_30785.txt'
Renamed '30790.txt' to 'fns30790_30790.txt'
Renamed '30795.txt' to 'fns30795_30795.txt'
Renamed '30796.txt' to 'fns30796_30796.txt'
Renamed '30800.txt' to 'fns30800_30800.txt'
Renamed '30808 (1).txt' to 'fns30808 (1)_30808 (1).txt'
Renamed '30813.txt' to 'fns30813_30813.txt'
Renamed '30816.txt' to 'fns30816_30816.txt'
Renamed '30815.txt' to 'fns30815_30815.txt'
Renamed '30819.txt' to 'fns30819_30819.txt'
Renamed '30817.txt' to 'fns30817_30817.txt'
Renamed '30820.txt' to 'fns30820_30820.txt'
Renamed '30822.txt' to 'fns30822_30822.txt'
Renamed '30821.txt' to 'fns30821_30821.txt'
Renamed '30904.txt' to 'fns30904_30904.txt'
Renamed '30830.txt' to 'fns30830_30830.txt'
Renamed '30849.txt' 

# **Renaming Gold Summary**

In [ ]:
import os

# Path to the directory containing the files
folder_path = "/content/drive/MyDrive/validation (2)/gold_summaries (1)"

# Iterate over all files in the directory
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # Check if the file is a text file
        # Extract the file number and part number from the filename
        file_number, part_number = filename.split('_')[0], filename.split('_')[1].split('.')[0]

        # Define the new filename pattern
        new_filename = f"fns{file_number}_{file_number}_{part_number}.txt"

        # Build the full paths for the old and new filenames
        old_filepath = os.path.join(folder_path, filename)
        new_filepath = os.path.join(folder_path, new_filename)

        # Rename the file
        os.rename(old_filepath, new_filepath)
        print(f"Renamed '{filename}' to '{new_filename}'")


Renamed '33130_2.txt' to 'fns33130_33130_2.txt'
Renamed '33070_2.txt' to 'fns33070_33070_2.txt'
Renamed '33090_3.txt' to 'fns33090_33090_3.txt'
Renamed '33090_1.txt' to 'fns33090_33090_1.txt'
Renamed '33097_3.txt' to 'fns33097_33097_3.txt'
Renamed '33090_2.txt' to 'fns33090_33090_2.txt'
Renamed '33097_2.txt' to 'fns33097_33097_2.txt'
Renamed '33097_1.txt' to 'fns33097_33097_1.txt'
Renamed '33090_4.txt' to 'fns33090_33090_4.txt'
Renamed '33098_1.txt' to 'fns33098_33098_1.txt'
Renamed '33098_2.txt' to 'fns33098_33098_2.txt'
Renamed '33098_3.txt' to 'fns33098_33098_3.txt'
Renamed '33155_4.txt' to 'fns33155_33155_4.txt'
Renamed '33155_2.txt' to 'fns33155_33155_2.txt'
Renamed '33130_3.txt' to 'fns33130_33130_3.txt'
Renamed '33155_1.txt' to 'fns33155_33155_1.txt'
Renamed '33130_1.txt' to 'fns33130_33130_1.txt'
Renamed '33155_3.txt' to 'fns33155_33155_3.txt'
Renamed '33154_1.txt' to 'fns33154_33154_1.txt'
Renamed '33154_2.txt' to 'fns33154_33154_2.txt'
Renamed '33154_3.txt' to 'fns33154_33154

# **Calculating ROuge Score**

In [19]:
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from rouge_score import rouge_scorer
import re

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def preprocess_summary(summary):
    summary = re.sub(r'\s+', ' ', summary)  # Remove extra whitespaces
    summary = summary.lower()
    summary = summary.translate(str.maketrans("", "", string.punctuation))
    summary = " ".join(summary.split())
    tokens = word_tokenize(summary)
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]
    processed_summary = " ".join(tokens)
    return processed_summary

def calculate_rouge_scores(generated_summary, actual_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    scores = scorer.score(generated_summary, actual_summary)

    return scores

# Example usage with actual and generated summaries
actual_summary_path = "/content/drive/MyDrive/validation (2)/gold_summaries (1)/fns31105_31105_1.txt"
with open(actual_summary_path, 'r', encoding='utf-8') as file:
    actual_summary = file.read()

# Assuming you have generated a summary
generated_summary = corrected_summary  # Replace this with your generated summary

# Preprocess both summaries
processed_generated_summary = preprocess_summary(generated_summary)
processed_actual_summary = preprocess_summary(actual_summary)

# Calculate ROUGE scores
rouge_scores = calculate_rouge_scores(processed_generated_summary, processed_actual_summary)

# Print ROUGE scores
print("ROUGE Scores:")
print(rouge_scores)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ROUGE Scores:
{'rouge1': Score(precision=0.5922574157868276, recall=0.13934232316063402, fmeasure=0.22560566886909894), 'rouge2': Score(precision=0.05482897384305835, recall=0.012894830237785402, fmeasure=0.02087922612776554), 'rougeL': Score(precision=0.12921065862242334, recall=0.03039981074047788, fmeasure=0.04921957291965911)}
